# Corretoras e Imobiliarias

Vamos pegar as tabelas de todos os corretores e imobiliarias do rio de janeiro

## Recuperando tabela de consultores/imobiliaris

### Importando Dependências

In [6]:
import time
import pandas as pd
import pyautogui
pyautogui.PAUSE = 1
import pyperclip
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from io import StringIO

### Lista de Bairros

In [ ]:
lista_bairros_zona_norte = [
    "Acari", "Anchieta", "Barros Filho", "Bento Ribeiro", "Brás de Pina", "Campinho", "Cavalcanti", "Cascadura", "Coelho Neto", "Colégio",
    "Complexo do Alemão", "Cordovil", "Costa Barros", "Engenheiro Leal", "Engenho da Rainha", "Guadalupe", "Honório Gurgel", "Inhaúma", "Irajá",
    "Jardim América", "Madureira", "Marechal Hermes", "Oswaldo Cruz", "Parada de Lucas", "Parque Anchieta", "Parque Colúmbia", "Pavuna", "Penha",
    "Penha Circular", "Quintinho Bocaiuva", "Ricardo de Albuquerque", "Rocha Miranda", "Tómas Coelho", "Turiaçu", "Vaz Lobo", "Vicente de Carvalho",
    "Vigário Geral", "Vila da Penha", "Vila Kosmos", "Vista Alegre" 
]
lista_bairros_ilha_gov_leopoldina = [
    "Bonsucesso", "Bancários", "Cacuia", "Cidade Universitária", "Cocotá", "Freguesia", "Galeão", "Jardim Carioca", "Jardim Guanabara", "Maré",
    "Moneró", "Olaria", "Pitangueiras", "Portuguesa", "Praia da Bandeira", "Ramos", "Ribeira", "Tauá", "Zumbi" 
]
lista_bairros_tijuca = [
    "Alto da Boa Vista", "Andaraí", "Grajaú", "Maracanã", "Praça da Bandeira", "Tijuca", "Vila Isabel", "Barra da Tijuca"
]
lista_bairros_meier = [
    "Abolição", "Agua Santa", "Cachambi", "Del Castilho", "Encantado", "Engenho de Dentro", "Engenho Novo", "Higienópolis", "Jacaré", "Jacarezinho",
    "Lins de Vasconcelos", "Manguinhos", "Maria da Graça", "Méier", "Piedade", "Pilares", "Riachuelo", "Rocha", "Sampaio", "São Francisco Xavier",
    "Todos os Santos",  
]

lista_bairros = lista_bairros_zona_norte + lista_bairros_ilha_gov_leopoldina + lista_bairros_tijuca + lista_bairros_meier
dict_bairros = {}

### Baixar tabela por bairro

In [ ]:
def baixar_tabela_por_bairro(bairro):
    url = 'https://www.crecirj.conselho.net.br/form_pesquisa_cadastro_geral_site.php'
    driver = webdriver.Edge()

    # Maximize a janela do navegador
    driver.maximize_window()
    
    #Abrir pagina
    driver.get(url)
    
    # Localiza os campos do formulário e preenche-os
    campo_bairro = driver.find_element("id", "input-27")
    campo_bairro.send_keys(bairro)

    # Envie o formulário
    campo_bairro.send_keys(Keys.RETURN)

    # Aguarde um tempo para a página carregar completamente
    time.sleep(4)

    # Pular para o final da pagina
    for _ in range(25):  
        pyautogui.press('down')  # Simula pressionar a tecla de seta para baixo
        time.sleep(0.2) 

    # Mova o cursor do mouse para o ponto médio do elemento usando PyAutoGUI
    pyautogui.moveTo(171, 979)

    # Simule um clique do mouse usando PyAutoGUI
    pyautogui.click()

    time.sleep(1)

    pyautogui.click()

    time.sleep(6)

    try:
        # Encontra a tabela na página
        table = driver.find_elements('css selector', 'table')
        table_html = table[0].get_attribute('outerHTML')
    except IndexError:
        print(f"Não existe nenhuma corretora/imobiliaria em {bairro}!")
        print(f"Verifique se o nome do bairro esta certo! {bairro}\n")
        driver.quit()
        return pd.DataFrame()
    else:
        df = pd.read_html(StringIO(table_html))[0]  # Este método retorna uma lista de DataFrames, portanto, pegamos o primeiro
        driver.quit()
        print(bairro, len(df),end='; ')
        return df

### Baixando as tabelas de todos os bairros

In [ ]:
def baixar_tabela_todos_bairros():
    for bairro in lista_bairros:
        tabela_bairro = baixar_tabela_por_bairro(bairro)
        dict_bairros.update({bairro:tabela_bairro})

#baixar_tabela_todos_bairros()

### Baixando apenas um item

In [ ]:
#tabela_bairro_tijuca = baixar_tabela_por_bairro("Tijuca")

#dict_bairros.update({"Tijuca":tabela_bairro_tijuca})

### Criando uma cópia para preservar o dicionário original

In [ ]:
dict_bairros_edit = dict_bairros.copy()
print(len(dict_bairros_edit))

### Removendo a natureza juridica do campo nome e adicionando campo bairro

In [ ]:
def editar_campos_empresa():
    for bairro, tb in dict_bairros_edit.items():
        try:
            tb.drop(columns=['Unnamed: 5'], inplace=True)
        except KeyError:
            pass
        
        for index, linha in tb.iterrows():
            nome = linha["Identificação"]
            tb.at[index, "Bairro"] = bairro
            
            if " PJ " in nome:
                novo_nome = nome.replace(" PJ P", "").replace(" PJ ", "")
                tb.at[index, "Identificação"] = novo_nome
                tb.at[index, "Natureza"] = "PJ P"
            elif " PF " in nome:
                novo_nome = nome.replace(" PF P", "").replace(" PF ", "")
                tb.at[index, "Identificação"] = novo_nome
                tb.at[index,"Natureza"] = "PF P"
        print("Bairro:", bairro)

#editar_campos_empresa()

### Preencher itens com "NAN" como ""

In [ ]:
def preencher_nan_com_vazio(df):
    # Iterar sobre todas as células do DataFrame
    for coluna in df.columns:
        for indice, valor in df[coluna].items():
            if pd.isna(valor):  # Verificar se o valor é NaN
                df.at[indice, coluna] = ""  # Substituir NaN por uma string vazia
    return df


new_dict = {}
for bairro, tb in dict_bairros_edit.items():
    novo_df = preencher_nan_com_vazio(tb)
    new_dict.update({bairro: novo_df})

dict_bairros_edit = new_dict.copy()

### Exibindo quantas corretoras/imobiliarias estão ativas e inativas

In [ ]:
ativos_qnt = 0
inativos_qnt = 0

def verifica_aitvos_inativos(dataframe):
    ativos = 0
    inativos = 0
    for index, linha in dataframe.iterrows():
        if "PJ" in  linha["Natureza"]:
            if linha["Situação"] == "ATIVO":
                ativos += 1 
            else: 
                inativos += 1
    return (ativos, inativos)

def contar_ativos_inativos_todos_bairros():
    for bairro, tb in dict_bairros_edit.items():
        novo_ativos_qnt, novo_inativos_qnt = verifica_aitvos_inativos(tb)
        ativos_qnt += novo_ativos_qnt
        inativos_qnt += novo_inativos_qnt

    print("Ativos", ativos_qnt)
    print("Inativos", inativos_qnt)
    print("Total", ativos_qnt + inativos_qnt)

#contar_ativos_inativos_todos_bairros()

### Filtrando Corretoras/Imobiliarias  (Situação = "ATIVO" e Certidão de Regularidade = "REGULAR")

In [ ]:
def bairros_ativos_disponiveis(bairros):
    novo_dict = {}
    for bairro_nome, bairro_tabela in bairros.items():
        print(bairro_nome)
        if not bairro_tabela.empty:
            nova_tabela = bairro_tabela.loc[bairro_tabela["Situação"] == "ATIVO"]
            nova_tabela = nova_tabela.loc[bairro_tabela["Certidão de Regularidade"] == "REGULAR"]
            novo_dict.update({bairro_nome:nova_tabela})
        else:
            print("Vazio",bairro_nome)
    return novo_dict.copy()

#print(len(dict_bairros_edit))
#dict_bairros_edit = bairros_ativos_disponiveis(dict_bairros_edit)
#print(len(dict_imobiliaria_ativa_disponivel["Tijuca"]))

## Descobrir CNPJ

#### Formatar CNPJ

In [ ]:
def formatar_cnpj(cnpj):
    cnpj_formatado = ""
    for i, char in enumerate(cnpj):
        if i == 1 or i == 4:
            cnpj_formatado += char + '.'
        elif i == 7:
            cnpj_formatado += char + '/'
        elif i == 11:
            cnpj_formatado += char + '-'
        else:
            cnpj_formatado += char
    return cnpj_formatado

### Verificar se o Edge esta maximizado

In [9]:
import pygetwindow

def verificar_edge_maximizado():
    # Obter todas as janelas abertas atualmente
    janelas = pygetwindow.getAllWindows()

    # Procurar a janela do Microsoft Edge entre as janelas abertas
    for janela in janelas:
        if "Microsoft​ Edge" in janela.title:
            # Verificar se a janela está maximizada
            return janela.isMaximized
    else:
        print("A janela do Microsoft Edge não está aberta.")
        return False

# Exemplo de uso
time.sleep(4)
verificar_edge_maximizado()


False

### Iniciando Edge

In [ ]:
def iniciando_edge_pyautogui(url):
    pyautogui.press("win")
    time.sleep(1)
    pyautogui.write("edge")
    time.sleep(1)
    pyautogui.press("enter")
    
    time.sleep(1)
    # Ativar o modo de tela cheia pressionando windows + up
    esta_maximizado = verificar_edge_maximizado()
    if not esta_maximizado:
        pyautogui.hotkey('win', 'up')

    time.sleep(2)
    pyautogui.write(url)  
    pyautogui.press("enter")
    time.sleep(3)

def iniciando_edge_selenium(url):
    driver = webdriver.Edge()

    # Maximize a janela do navegador
    driver.maximize_window()
    
    #Abrir pagina
    driver.get(url)
    time.sleep(3)
    return driver


### Sites que realizaremos as buscas

In [ ]:
URL_LINKANA = "https://cnpj.linkana.com/"
URL_CNPJBIZ = "https://cnpj.biz/empresas"
URL_ECONODATA = "https://www.econodata.com.br/consulta-empresa"

### Recuperando String de Print da Tela

In [ ]:
import easyocr
from PIL import Image
import logging

# Configura o nível de logging para suprimir os avisos da EasyOCR
logging.getLogger('easyocr').setLevel(logging.ERROR)

def recuperando_string_print(x1, y1, width, height):
    '''
    Captura um print da tela e transforma em string
    '''
    # Inicializa o EasyOCR para o idioma português
    reader = easyocr.Reader(['pt'])

    # Captura uma região da tela onde o texto está localizado (ajuste as coordenadas conforme necessário)
    screenshot = pyautogui.screenshot(region=(x1, y1, width, height))

    # Salva a captura de tela em um arquivo temporário
    screenshot_path = "screenshot_temp.png"
    screenshot.save(screenshot_path)

    # Usa EasyOCR para realizar OCR na imagem
    resultados = reader.readtext(screenshot_path)
    
    # Extrai o texto dos resultados
    texto_copiado = ""
    for resultado in resultados:
        texto_copiado += resultado[1] + " "
    return texto_copiado.strip()



### Verificar CNPJ no CRESCI - RJ

In [ ]:
def verificar_cnpj_cresci(cnpj, inscricao_empresa):
    # empresa = {informações da linha do dataframe}
    # Entrar no Cresci
    driver = iniciando_edge_selenium('https://www.crecirj.conselho.net.br/form_pesquisa_cadastro_geral_site.php')
    # Digitar o CNPJ
    time.sleep(1)
    campo_cnpj = driver.find_element("id", "input-21")
    time.sleep(1)
    campo_cnpj.send_keys(cnpj)
    time.sleep(1)
    # Envie o formulário
    campo_cnpj.send_keys(Keys.RETURN)
    time.sleep(2)
    # Verificar se é compativel
    try:
        # Encontra a tabela na página
        table = driver.find_elements('css selector', 'table')
        table_html = table[0].get_attribute('outerHTML')
    except IndexError:
        driver.quit()
        return False
    else:
        df = pd.read_html(StringIO(table_html))[0]  # Este método retorna uma lista de DataFrames, portanto, pegamos o primeiro
        driver.quit()
        # Recuperando dados da empresa
        empreasa_info = df.iloc[0]
        empresa_inscricao = empreasa_info["Inscrição"]

        if int(empresa_inscricao) == int(inscricao_empresa):
            return True
        else:
            print(f"A inscriçãoda da empresa {inscricao_empresa} é diferente de {empresa_inscricao}")
            return False


### Buscar no CNPJ.BIZ

#### Pesquisar empresa pelo nome

In [ ]:
def pesquisar_empresa_nome_cnpjbiz(nome_empresa):
    '''
    Pesquisa uma empresa pelo nome
    '''
    time.sleep(2)
    pyperclip.copy(nome_empresa)
    time.sleep(1)
    pyautogui.click(1062, 409)
    pyautogui.hotkey("ctrl", "v")
    time.sleep(1)
    pyautogui.press("enter")
    time.sleep(1)
    

#### Verificar se existe empresa

In [ ]:
def verificar_existe_empresa_cnpjbiz():
    '''
    Verifica se ao pesquisar a empresa, sera exibido uma mensagem de erro
    Caso não seja encontrado nada sera retornado verdadeiro, caso o contrario falso
    '''
    deu_ruim = recuperando_string_print(639, 235, 500, 300)
    time.sleep(1)
    if "Opa! Alguma coisa saiu errado" in deu_ruim:
        print("Deu ruim")
        return False
    else:
        return True

#### Verificar se a empresa é do Rio

In [ ]:
def verifica_empresa_no_rj_cnpjbiz():
    regiao = recuperando_string_print(639, 430, 300, 100)
    time.sleep(1)
    return "Rio de Janeiro/RJ" in regiao

#### Capturar CNPJ da empresa

In [ ]:
def capturar_cnpj_empresa_cnpjbiz():
    '''
    Captura um print da tela onde o cnpj deve estar. 
    E transforma essa imagem em texto
    '''

    pyautogui.moveTo(445,430)
    texto = recuperando_string_print(445, 430, 300, 200).split(" ")[0]
    return texto

#### Mover ate empresa

In [ ]:
def mover_ate_empresa_cnpjbiz():
    pyautogui.PAUSE = .5
    for _ in range(5):
        pyautogui.press("down")
    pyautogui.PAUSE = 1

#### Acessa a Pagina da empresa e inicia um driver no selenium com essa url

In [ ]:
def acessando_empresas_info_to_selenium_cnpjbiz():
    '''
    Acessar o site com as informações da empresa e criar um driver no selenium com essa mesma url
    '''
    pyautogui.click(445,430)
    time.sleep(1)
    pyautogui.click(581,62)
    time.sleep(1)
    pyautogui.hotkey("ctrl", "c")

    time.sleep(1)
    link = pyperclip.paste()
   
    pyautogui.click(1888, 18)
    return iniciando_edge_selenium(link)
    

#### Acessando pagina da empresa e descendo para a area de contatos e endereco

In [ ]:
def abrindo_empresa_cnpjbiz():
    pyautogui.click(445,430)

def direcionando_contatos_endereco_cnpjbiz():
    pyautogui.PAUSE = .5
    for _ in range(9):
        pyautogui.press("down")
    pyautogui.PAUSE = 1

#direcionando_contatos_endereco_cnpjbiz()

#### Recuperar Nome Fantasia

In [ ]:
def direcionando_nome_fantasia_cnpjbiz():
    time.sleep(4)
    pyautogui.PAUSE = .5
    for _ in range(6):
        pyautogui.press("down")
    pyautogui.PAUSE = 1


def recuperar_nome_fantasia_cnpjbiz():
    nome_fantasia_texto = recuperando_string_print(0, 0, 1000, 1000).replace(":", "")
    try:
        nome_fantasia_texto = nome_fantasia_texto.split(" Nome Fantasia ")[1].split(" Data da Abertura ")[0]
        return nome_fantasia_texto
    except:
        return "Nenhum Nome Fantasia encontrado"

#recuperar_nome_fantasia_cnpjbiz()
    


#### Recuperando Contatos

In [ ]:
def recuperando_contatos_cnpj_biz(contatos_texto_og):
    # Digitar o CNPJ
    time.sleep(2)
    contatos=""
    try:
        contatos_texto = contatos_texto_og.split("Contatos ")[1].split(" Localização ")[0]
        contatos_texto = contatos_texto.replace("(Ligar)", "")
        contatos_texto = contatos_texto.replace("Telefone(s): ", "")
        contatos_texto = contatos_texto.replace(")", "")
        contatos_texto = contatos_texto.replace("(", "")
        contatos_texto = contatos_texto.replace("E-mails", "")
        contatos_texto = contatos_texto.replace("E-mail:", "")
        
        contatos_texto = contatos_texto.replace(" Enviar Email", "")
        contatos_texto = contatos_texto.replace(" Enviar E-mail", "")
        contatos_texto = contatos_texto.replace(" Whatsapp", "")
        contatos_texto = contatos_texto.replace(". ", ".") 

        contatos = contatos_texto
    except:
        contatos = ""
    return contatos

#recuperando_contatos_cnpj_biz()

#### Recuperando Socios

In [ ]:
def recuperando_socios_cnpj_biz():
    pyautogui.PAUSE = .5
    for _ in range(12):
        pyautogui.press("down")
    pyautogui.PAUSE = 1
    socios_texto = recuperando_string_print(0, 0, 1000, 1000)
    try:
        socios_texto = socios_texto.split("Quadro de Sócios e Administradores")[1].split("Qualificação do responsável pela empresa: ")[0]
        socios_texto = socios_texto.replace(" Sócio ", " - ")
        socios_texto = socios_texto.replace(" Sócio-Administrador ", " - ")

        return socios_texto
    except:
        return "Nenhum Socio Encontrado"
    
#recuperando_socios_cnpj_biz()


#### Recuperando Endereco

In [ ]:
def recuperando_endereco_cnpj_biz(endereco_texto_og):
    try:
        endereco_texto = endereco_texto_og.split("Para correspondência:")[1].split("Procurar no Google Maps")[0]
        print(endereco_texto)

        return endereco_texto
    except:
        return "Nenhum Socio Encontrado"
#recuperando_endereco_cnpj_biz()

#### Listar elementos html !!!!!!!!!!!!!!!!!!

In [ ]:
def listar_elementos_pagina(driver:webdriver.Edge, elemento):
    elementos = driver.find_elements(By.XPATH, f"following-sibling::{elemento}")
    for idx, element in enumerate(elementos):
        print(idx, element.text)
    else:
        print(f"Não possui nenhum elemento <{elemento}/>")
    return elementos

#### Buscar dados no CNPJ.Biz

In [ ]:
def buscar_empresa_cnpjbiz(nome_empresa, inscricao_empresa):
    pyautogui.PAUSE = 1
    iniciando_edge_pyautogui(URL_CNPJBIZ)
    time.sleep(2)
    pesquisar_empresa_nome_cnpjbiz(nome_empresa)
    
    existe_empresa = verificar_existe_empresa_cnpjbiz()
    
    if existe_empresa:
        mover_ate_empresa_cnpjbiz()
        for i in range(10):
            # Verificar se é no RJ
            empresa_uf_RJ = verifica_empresa_no_rj_cnpjbiz()

            if empresa_uf_RJ:
                cnpj = capturar_cnpj_empresa_cnpjbiz()
                cnpj_valido = verificar_cnpj_cresci(cnpj, inscricao_empresa)
            
                if cnpj_valido:
                    abrindo_empresa_cnpjbiz()
                    direcionando_nome_fantasia_cnpjbiz()
                    nome_fantasia = recuperar_nome_fantasia_cnpjbiz()
                    direcionando_contatos_endereco_cnpjbiz()

                    contatos_endereco_texto_og = recuperando_string_print(0, 0, 1000, 1000)
                    contatos = recuperando_contatos_cnpj_biz(contatos_endereco_texto_og)
                    endereco = recuperando_endereco_cnpj_biz(contatos_endereco_texto_og)
                    socios = recuperando_socios_cnpj_biz()
                    print("Nome: ", nome_empresa)
                    dados_novos = {"Contatos":contatos, "Socios": socios, "Endereço": endereco, "CNPJ": cnpj, "Nome Fantasia":nome_fantasia}
                    pyautogui.click(1888, 18)
                    return dados_novos
                else:
                    pyautogui.press("down")
            else:
                pyautogui.press("down")
    else:
        pyautogui.click(1888, 18)
        return {"error": "Nenhuma empresa encontrada"}
        


    pyautogui.click(1888, 18)
    return False
    
#cnpj_empresa = buscar_empresa_cnpjbiz("AM IMOVEIS CONSULTORIA LTDA", "8170")

### Buscar no Linkana

#### Buscar no Linkana pelo nome da Empresa

In [ ]:
def pesquisar_empresa_nome_linkana(nome_empresa, driver):
    campo_nome = driver.find_element(By.ID, "q")
    campo_nome.send_keys(nome_empresa)
    campo_nome.send_keys(Keys.RETURN)

#### Verificando se existe empresa

In [ ]:
def verificar_existe_empresa_linkana():
    texto = recuperando_string_print(313, 529, 500, 70)
    return not "Nenhum resultado encontrado" in texto
    

#### Verifica se a empresa é no RJ

In [ ]:
def verifica_empresa_no_rj_linkana():
    texto = recuperando_string_print(1119, 621, 100, 70)
    return "RJ" in texto

#### Recuperando Nome Fantasia

In [ ]:
def recuperando_nome_fantasia_linkana(driver:webdriver.Edge):
    lista = driver.find_elements(By.TAG_NAME, "ul")[0].text
    nome_fantasia = lista.split("\nNome fantasia\n")[1].split("\nSituação cadastral\n")[0]
    return nome_fantasia



#### Recuperando Contato

In [ ]:
def recuperando_contatos_linkana(driver:webdriver.Edge):
    lista = driver.find_elements(By.TAG_NAME, "ul")[0].text
    email = lista.split("\nE-mail\n")[1].split("\nTelefone\n")[0]
    telefone = ""
    try:
        telefone = lista.split("\nTelefone\n")[1]
    except:
        pass
    
    contato = f"{email}, {telefone}"
    return contato

#### Recuperando Socios

In [ ]:
def recuperando_socios_linkana(driver:webdriver.Edge):
    lista = driver.find_elements(By.TAG_NAME, "ul")[1].text.split("\n")
    socios = ""
    for idx, item in enumerate(lista):
        if idx % 2 == 0:
            socios += f" {item} -"
    return socios

#### Recuperando Endereco

In [ ]:
def recuperando_endereco_linkana(driver:webdriver.Edge):
    lista = driver.find_elements(By.TAG_NAME, "ul")[3].text.split("\n")
    endereco = ""
    for idx, item in enumerate(lista):
        if idx % 2 != 0:
            endereco += f" {item} -"
    return endereco

#### Buscar Empresa Linkana

In [ ]:
def buscar_empresa_linkana(nome_empresa, inscricao_empresa):
    driver = iniciando_edge_selenium(URL_LINKANA)
    
    pesquisar_empresa_nome_linkana(nome_empresa, driver)

    existe_empresa = verificar_existe_empresa_linkana()
    if existe_empresa:
        empresa_uf_rj = verifica_empresa_no_rj_linkana()
        if empresa_uf_rj:
            cnpj = recuperando_string_print(337, 584, 250, 60).split(" ")[1]
            cnpj_valido = verificar_cnpj_cresci(cnpj, inscricao_empresa)
            if cnpj_valido:
                cnpj = formatar_cnpj(cnpj)
                pyautogui.click(337,584)
                time.sleep(3)
                nome_fantasia = recuperando_nome_fantasia_linkana(driver)
                contatos = recuperando_contatos_linkana(driver)
                socios = recuperando_socios_linkana(driver)
                enderecos = recuperando_endereco_linkana(driver)
                driver.quit()    
                return {"Contatos":contatos, "Socios": socios, "Endereço": enderecos, "CNPJ":cnpj, "Nome Fantasia":nome_fantasia}    
    

    driver.quit()    
    return False

#buscar_empresa_linkana("2F IMOBILIARIA LTDA", "10591")

In [ ]:
def teste_2(nome_empresa):
    driver = iniciando_edge_selenium(URL_LINKANA)
    
    pesquisar_empresa_nome_linkana(nome_empresa, driver)

    existe_empresa = verificar_existe_empresa_linkana()
    if existe_empresa:
        empresa_uf_rj = verifica_empresa_no_rj_linkana()
        if empresa_uf_rj:
            cnpj = recuperando_string_print(337, 584, 250, 60).split(" ")[1]
            pyautogui.click(337,584)
            time.sleep(3)
            contatos = recuperando_contatos_linkana(driver)
            socios = recuperando_socios_linkana(driver)
            enderecos = recuperando_endereco_linkana(driver)
            driver.quit()    
            return {"Contatos":contatos, "Socios": socios, "Endereço": enderecos}    
#teste_2("ACTA CORRETORA DE IMOVEIS LTDA")
#teste_2("2F IMOBILIARIA LTDA")
    

### Buscar no CNPJ INFO

#### Pesquisar empresa no cnpj info

In [ ]:
def pesquisar_empresa_nome_cnpjinfo(nome_empresa, driver):
    campo_empresa = driver.find_elements(By.CSS_SELECTOR, "h3 input")[0]
    campo_empresa.send_keys(nome_empresa)
    campo_empresa.send_keys(Keys.RETURN)

#### Recuperar CNPJ no cnpj info

In [ ]:
def recuperar_cnpj_cnpjinfo(driver):
    content = driver.find_element(By.ID, "content")
    lista_empresas = content.find_elements(By.TAG_NAME, "ul")[0]
    itens_lista_empresas = lista_empresas.find_elements(By.TAG_NAME, "li")
    primeiro_item = itens_lista_empresas[0]
    cnpj = primeiro_item.find_elements(By.TAG_NAME, "a")[0]
    return cnpj


#### Recuperar endereço

In [ ]:
def recuperando_endereco_cnpjinfo(driver):
    content = driver.find_element(By.ID, "content")
    content_text = content.text
    endereco = content_text.split("\nEndereço\n")[1].split("\nContatos\n")[0]
    print(endereco)
    return endereco

#### Recuperar contatos

In [ ]:
def recuperando_contatos_cnpjinfo(driver:webdriver.Edge):
    content = driver.find_element(By.ID, "content").text
    contatos = content.split("\nTelefone(s): ")[1].split("\nSócios\n")[0].strip().split(" e ")
    print(contatos)
    return contatos
    

#### Recuperando Socios

In [ ]:
def recuperando_socios_cnpjinfo(driver:webdriver.Edge):
    content = driver.find_element(By.ID, "content").text
    socios = content.split("Sócios\nCódigo Nome Data de entrada Qualificação\n")[1].split("Atividades de negócios da empresa")[0].split("\n")
    print(socios)

####

In [ ]:
def buscar_empresa_cnpjinfo(nome_empresa, inscricao_empresa):
    driver = iniciando_edge_selenium("http://cnpj.info/")
    pesquisar_empresa_nome_cnpjinfo(nome_empresa, driver)
    cnpj = recuperar_cnpj_cnpjinfo(driver)
    cnpj_valido = verificar_cnpj_cresci(cnpj.text, inscricao_empresa)
    if cnpj_valido:
        cnpj.click()
        endereco = recuperando_endereco_cnpjinfo(driver)
        contatos = recuperando_contatos_cnpjinfo(driver)
        socios = recuperando_socios_cnpjinfo(driver)
        return {"Contatos":contatos, "Socios": socios, "Endereço": endereco}       
    else:
        print("CNPJ INVALIDO")

    return False

#buscar_empresa_cnpjinfo("2M INTERMEDIAÇÕES LTDA", "10663")

In [ ]:
def teste():
    driver = iniciando_edge_selenium("http://cnpj.info/24701445000167")
    time.sleep(5)
    #content = driver.find_element(By.ID, "content")
    recuperando_socios_cnpjinfo(driver)
    #print(content.text)

#teste()

### Realizar Busca Completa

#### Listas Empresas

In [ ]:
lista_empresas_informacoes_validas = []
lista_empresas_sem_informacoes = {}

#### Buscar empresa

In [1]:
def buscar_dados_empresas_linkana(empresa):
    nome_empresa = empresa["Identificação"]
    inscricao_empresa = empresa["Inscrição"]

    dados_novos = {}
    recuperou = buscar_empresa_linkana(nome_empresa, inscricao_empresa)
    if not recuperou or "error" in recuperou:
        if "error" in recuperou:
            dados_novos.update(recuperou)

        print("Não Achou: ", nome_empresa)
    else:
        dados_novos.update(recuperou)

    return dados_novos


In [2]:
def buscar_dados_empresas_cnpjbiz(empresa):
    nome_empresa = empresa["Identificação"]
    inscricao_empresa = empresa["Inscrição"]

    dados_novos = {}
    recuperou = buscar_empresa_cnpjbiz(nome_empresa, inscricao_empresa)

    print("Recuperou", recuperou)
    if not recuperou or "error" in recuperou:
        if "error" in recuperou:
            dados_novos.update(recuperou)

        print("Não Achou: ", nome_empresa)
    else:
        dados_novos.update(recuperou)

    print("DAdos Novos", dados_novos)
    
    return dados_novos

In [ ]:
def transformando_lista_string(lista):
    if type(lista) == type([]):
        nova_lista = []
        for item in lista:
            if type(item) == type([]):
                nova_lista.append(", ".join(item))
            else:
                nova_lista.append(item)
        return " | ".join(nova_lista)

In [ ]:
def buscar_dados_empresa(empresa):
    try:
        dados_novos = buscar_dados_empresas_linkana(empresa)
    except Exception as e:
        print(e)
        return pd.DataFrame()
    else:
        print(dados_novos)
        if len(dados_novos) > 0:
            dados_empresa = dict(empresa)
            for idx_item, valor in dados_novos.items():
                #valor_str = transformando_lista_string(valor)
                dados_empresa.update({idx_item: valor})
            return pd.DataFrame(dados_empresa, index=[0])
            


In [ ]:
def buscar_todas_empresas_bairro(df:pd.DataFrame):
    completos_columns = ["Inscrição","Identificação","CNPJ","Nome Fantasia","Situação","Certidão de Regularidade","Contato","Natureza","Bairro","Contatos","Socios","Endereço"]
    completos_df = pd.DataFrame(columns=completos_columns)
    incompletos_columns = ["Inscrição","Identificação","Situação","Certidão de Regularidade","Contato","Natureza","Bairro","error"]
    incompletos_df = pd.DataFrame(columns=incompletos_columns)

    for idx, empresa in df.iterrows():
        try:
            empresa_completa = buscar_dados_empresa(empresa)
            display(empresa_completa)
            if not empresa_completa.empty and not "error" in empresa_completa:
                completos_df = pd.concat([completos_df, empresa_completa], ignore_index=True)
            elif "error" in empresa_completa:
                incompletos_df = pd.concat([incompletos_df, empresa_completa], ignore_index=True)
                print("DEu erro: ", idx)
        except Exception as e:
                print("DEu erro: ", idx, e)
    if "error" in incompletos_df:
        incompletos_df.drop(columns="error")
                

    return completos_df, incompletos_df

## Conectar Google Sheets 

### Importando dependências

In [ ]:
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from gspread_dataframe import set_with_dataframe
import gspread

### Criar Planilhas no GOOGLE SHEETS

In [ ]:
def criar_planilha_gs(nome, service):
    # Create the spreadsheet
    planilha = {"properties": {"title": nome}}
    planilha = (
        service.spreadsheets()
        .create(body=planilha, fields="spreadsheetId")
        .execute()
    )
    spreadsheet_id = planilha.get("spreadsheetId")
    print(f"Spreadsheet ID: {spreadsheet_id}")
    return spreadsheet_id

### Ler Planilhas no GOOGLE SHEETS

In [ ]:
def ler_planilha(id_planilha, service, range="Página1!A:Z"):
    planilha = service.spreadsheets().values().get(spreadsheetId=id_planilha, range=range).execute()
    print(planilha['values'])
    return planilha

### Editar planilhas no GOOGLE SHEETS

In [ ]:
def editar_planilha(id_planilha, service, novo_conteudo, range="Pagina1!A"):
    planilha = service.spreadsheets().values().update(
        spreadsheetId=id_planilha,
        range=range, valueInputOption="USER_ENTERED",
        body={'values':novo_conteudo}
    ).execute()
    return planilha

### Transforma Dataframe em Lista

In [ ]:
def transforma_dataframe_em_lista(dataframe:pd.DataFrame, add_columns=False):
    nova_lista = []
    columns = list(dataframe.columns)
    if add_columns:
        nova_lista.append(columns)
    for index, linha in dataframe.iterrows():
        itens_lista = []
        for item in linha:
            itens_lista.append(item)
        nova_lista.append(itens_lista)
    return nova_lista

### Validar Token GOOGLE SHEETS

In [ ]:
SCOPES = ["https://www.googleapis.com/auth/drive", "https://www.googleapis.com/auth/spreadsheets"]

def validar_token_google_sheets():
  creds = None
  # The file token.json stores the user's access and refresh tokens, and is
  # created automatically when the authorization flow completes for the first
  # time.
  if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)
  # If there are no (valid) credentials available, let the user log in.
  if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
      creds.refresh(Request())
    else:
      flow = InstalledAppFlow.from_client_secrets_file(
          "client_secret.json", SCOPES
      )
      creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open("token.json", "w") as token:
      token.write(creds.to_json())
  return creds

### Transformar Dataframe em planilha

In [ ]:
# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/drive", "https://www.googleapis.com/auth/spreadsheets"]
id_planilha = "1AeDuHW7f4-7vBVCgQvMBYqx2ZCTxcO7D0qbzMZ2l8Ps"

def transformar_dataframe_planilha(title, dataframe):
    creds = validar_token_google_sheets()
    ordem_df = ["Inscrição", "Identificação", "Situação", "Certidão de Regularidade", "Contato", "Natureza", "Bairro"]	
    if all(coluna in dataframe.columns for coluna in ordem_df):
        dataframe = dataframe[ordem_df]																	
        service = build("sheets", "v4", credentials=creds)
        conteudo_para_adicionar = transforma_dataframe_em_lista(dataframe, add_columns=True)
        editar_planilha(id_planilha, service, conteudo_para_adicionar, range=f"{title}!A1:Z")

    print(f"Planilha Editada - {title}")

In [ ]:
#for bairro, tb in dict_bairros_edit.items():
    #transformar_dataframe_planilha(bairro, tb)

### Recuperar planilha bairros

In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd

def baixar_pagina_planilha(nome_planilha, nome_pagina):
  creds = validar_token_google_sheets()
    # Substitua pelo nome do seu arquivo de credenciais

  # Autorize e abra a planilha
  client = gspread.authorize(creds)
  spreadsheet = client.open(nome_planilha)  # Substitua pelo nome da sua planilha
  worksheet = spreadsheet.worksheet(nome_pagina)  # Substitua "Tijuca" pelo nome da sua página

  # Obtenha os dados da planilha como uma lista de dicionários
  data = worksheet.get_all_records()

  # Converta os dados em um DataFrame do Pandas
  df = pd.DataFrame(data)

  # Agora você pode usar df para trabalhar com seus dados
  return df

In [ ]:
def baixar_todos_bairros_planilha(lista_paginas, nome_planilha):
    bairros_planilha = {}
    for pagina in lista_paginas:
        df = baixar_pagina_planilha(nome_planilha, pagina)
        bairros_planilha.update({pagina:df})
        time.sleep(2)
    print("Finalizado")
    return bairros_planilha

#dict_bairros_edit = baixar_todos_bairros_planilha(lista_bairros, "Bairros PJ")


### Remover PF

In [ ]:
def remover_empresas_pf_inativas(df):
    if len(df) > 0:
        novo_df = df[df['Natureza'].str.contains('PJ') == True].copy()
        novo_df = novo_df[novo_df['Situação'] == "ATIVO"].copy()
        return novo_df
    else:
        return pd.DataFrame(columns=["Inscrição", "Identificação", "Situação", "Certidão de Regularidade", "Contato", "Natureza", "Bairro"])


### Atualizar Planilha Google Sheets

In [ ]:
def remover_planilha_adicionar_df(nome_planilha, nome_pagina, novo_df):
    creds = validar_token_google_sheets()
    client = gspread.authorize(creds)
    spreadsheet = client.open(nome_planilha)
    worksheet = spreadsheet.worksheet(nome_pagina)

    # Limpar os dados na planilha
    worksheet.clear()
    # Atualizar a planilha com os dados atualizados
    novo_df_list = transforma_dataframe_em_lista(novo_df, add_columns=True)
    worksheet.append_rows(novo_df_list)
    print("Planilha atualizada")

##

In [ ]:
for bairro in lista_bairros:
    break
    df_novo = baixar_pagina_planilha("Bairros", bairro)
    df_novo = remover_empresas_pf_inativas(df_novo)
    display(bairro, len(df_novo))

    remover_planilha_adicionar_df("Bairros PJ", bairro, df_novo)
    time.sleep(2)

In [214]:
tijuca_df = baixar_pagina_planilha("Bairros PJ", "Tijuca")
tijuca_completos_df = baixar_pagina_planilha("Bairros Completos", "Tijuca")
tijuca_incompletos_cnpjbiz_df = baixar_pagina_planilha("Empresas Negativas CNPJ BIZ", "Tijuca")
tijuca_completos_cnpjbiz_df = baixar_pagina_planilha("Empresas CNPJBIZ", "Tijuca")


# Filtrar linhas em df_a que não estão presentes em df_b
result_df = tijuca_df[~tijuca_df['Inscrição'].isin(tijuca_completos_df['Inscrição'])]
#result_df = result_df[~result_df['Inscrição'].isin(tijuca_incompletos_cnpjbiz_df['Inscrição'])]
result_df = result_df[~result_df['Inscrição'].isin(tijuca_completos_cnpjbiz_df['Inscrição'])]

print(len(result_df))
#time.sleep(2)
#tijuca_final_df = buscar_todas_empresas_bairro(result_df)


19


In [ ]:
#tijuca_final_df = buscar_todas_empresas_bairro(result_df)
result_df_5 = tijuca_completos_df.head(1)

#display(tijuca_incompletos_cnpjbiz_df)

completos_tijuca, incompletos_tijuca = buscar_todas_empresas_bairro(tijuca_completos_df)

#display(len(tijuca_completos_df))

#display((tijuca_final_df))

#display((lista_empresas_sem_informacoes["Tijuca"]))
#tijuca_final_df = buscar_todas_empresas_bairro(result_df)

In [215]:
#display((completos_tijuca), len(incompletos_tijuca))
#display((completos_tijuca), (incompletos_tijuca))
concat_empresas_completas = pd.concat([tijuca_completos_df, tijuca_completos_cnpjbiz_df], ignore_index=True)
#concat_empresas_incompletas = pd.concat([incompletos_tijuca, tijuca_incompletos_cnpjbiz_df], ignore_index=True)
#display(len(concat_empresas_completas), len(concat_empresas_incompletas))
'''
def filtrar_texto(txt):
    return txt.split("Situação cadastral\n")[0]

tijuca_completos_df['Nome Fantasia'] = tijuca_completos_df['Nome Fantasia'].apply(lambda x: filtrar_texto(x))
display(tijuca_completos_df)
'''


'\ndef filtrar_texto(txt):\n    return txt.split("Situação cadastral\n")[0]\n\ntijuca_completos_df[\'Nome Fantasia\'] = tijuca_completos_df[\'Nome Fantasia\'].apply(lambda x: filtrar_texto(x))\ndisplay(tijuca_completos_df)\n'

In [216]:
#am_empresa = result_df[result_df["Inscrição"] == 8170].iloc[0]
#remover_planilha_adicionar_df("Empresas CNPJBIZ", "Tijuca", concat_empresas_completas)
remover_planilha_adicionar_df("Bairros Completos", "Tijuca", concat_empresas_completas)

Planilha atualizada


In [ ]:
#am_empresa = buscar_dados_empresa(am_empresa)
#display(am_empresa)
display(tijuca_incompletos_cnpjbiz_df)


Baixar empresas que eu nao consegui baixar

remover pos fixo do nome

In [ ]:
def remover_pos_fixo_df(df:pd.DataFrame):
    df_novo = pd.DataFrame()
    for idx, item in df.iterrows():
        item_dict = dict(item)
        nome_antigo = item_dict["Identificação"]
        nome_novo = nome_antigo.replace("LTDA", "")
        nome_novo = nome_novo.replace("EIRELI", "")
        nome_novo = nome_novo.replace("S/A", "")
        nome_novo = nome_novo.replace(".", "")
        nome_novo = nome_novo.replace("-", "")
        nome_novo = nome_novo.replace("ME", "")
        nome_novo = nome_novo.replace("VICEVERSA", "VICE-VERSA")
        item_dict["Identificação"] = nome_novo
        item_df = pd.DataFrame(item_dict, index=[0])
        df_novo = pd.concat([df_novo, item_df], ignore_index=True)
    return df_novo
    
tijuca_incompletos_cnpjbiz_df = remover_pos_fixo_df(result_df)


In [ ]:
def restaurar_identificacao(df_novo: pd.DataFrame, df_antigo: pd.DataFrame, id_col: str = 'Inscrição') -> pd.DataFrame:
    df_novo_restaurado = df_novo.copy()  # Criando uma cópia do dataframe novo para evitar modificações indesejadas
    
    # Criando um dicionário de mapeamento entre os IDs do dataframe novo e as identificações do dataframe antigo
    mapeamento = dict(zip(df_antigo[id_col], df_antigo['Identificação']))
    
    # Restaurando os valores originais da coluna "Identificação" no dataframe novo com base no ID
    df_novo_restaurado['Identificação'] = df_novo[id_col].map(mapeamento)
    
    return df_novo_restaurado

In [ ]:
tijuca_incompletos_cnpjbiz_df = baixar_pagina_planilha("Empresas Negativas CNPJ BIZ", "Tijuca")
tijuca_completos_cnpjbiz_df = baixar_pagina_planilha("Empresas CNPJBIZ", "Tijuca")
tijuca_completos_cnpjbiz_df_2 = restaurar_identificacao(completos_tijuca, tijuca_incompletos_cnpjbiz_df, id_col="Inscrição")


In [ ]:
incompletos_tijuca = incompletos_tijuca.drop(columns=["error"])
#display(tijuca_completos_cnpjbiz_df_2)
concat_empresas_completas = pd.concat([tijuca_completos_cnpjbiz_df_2, tijuca_completos_cnpjbiz_df], ignore_index=True)
#display(len(concat_empresas_completas))

